#####Notebook for Data exploration

In [2]:
import numpy as np
import sklearn
import csv
from sklearn.feature_extraction import DictVectorizer

In [3]:
train_file = "WaterPump-training-values.csv"
train_labels = "WaterPump-training-labels.csv"
test_file = "WaterPump-test-values.csv"

def getData(lines=None, step=1, fileName="WaterPump-training-values.csv"):
    #method to import part or all of data from CSV into list
    #lines: how many lines to read; None means read to end of file
    #step: step size, so step=2 would read every 2nd line
    reader = csv.DictReader(open(fileName, 'rU'))
    result = []
    
    i=0
    #read to end of file, skipping lines if necessary
    if lines==None:
        for row in reader:
            if i%step==0:
                result.append(row)
            i += 1
    
    #read until specified number of lines are stored
    for row in reader:
        if i>=lines*step:
            break
        if i%step==0:
            result.append(row)
        i += 1
    
    return result

In [6]:
def intDate(date):
    #convert date string (2011-03-04) to int
    dates = date.split('-')
    #this is approximate formula, ignoring leap years and variance in month lengths
    return int(dates[0])*365 + int(dates[1])*30 + int(dates[2])

def quantData(data):
    #getData() reads everything as string, so this changes appropriate continuous variables to ints/floats
    #mutates given data
    #TODO: how to deal with dates? ('date_recorded')
    for row in data:
        row['longitude'] = float(row['longitude'])
        row['latitude'] = float(row['latitude'])
        row['gps_height'] = int(row['gps_height'])
        #these are categorical data and should not be converted
        #row['region_code'] = int(row['region_code'])
        #row['district_code'] = int(row['district_code'])
        row['amount_tsh'] = float(row['amount_tsh'])
        row['population'] = float(row['population'])
        row['construction_year'] = float(row['construction_year'])
        row['num_private'] = int(row['num_private'])  #what the heck is this??
        row['permit'] = row['permit'] == 'TRUE'
        row['public_meeting'] = row['public_meeting'] == 'TRUE'
        row['date_recorded'] = intDate(row['date_recorded'])
        
        #removing columns
        del row['recorded_by']  #only 1 unique value
        del row['quantity_group'] #redundant with 'quantity'
        del row['source_type']
        del row['waterpoint_type_group']
        del row['payment_type']
        #need to confirm the following is redundant with 'region'
        #del row['region_code']
        
        #these were dropped in the example: http://community.drivendata.org/t/share-your-approach/65
        #because they had too many levels
        del row['subvillage']
        del row['wpt_name']
        del row['num_private']
        del row['ward']
        del row['scheme_name']
        del row['installer']
        del row['funder']
        #removed because they are proxies
        del row['region_code']
        del row['district_code']
        del row['lga']
        del row['scheme_management']
    return data

def vectorizeData(data):
    #vectorize data, data should be dictionary
    #http://nbviewer.ipython.org/gist/sarguido/7423289
    vec = DictVectorizer()
    vectData = vec.fit_transform(data).toarray()
    return vectData

In [4]:
rawData = getData(5)
print rawData[:1]

[{'extraction_type': 'gravity', 'region_code': '11', 'gps_height': '1390', 'recorded_by': 'GeoData Consultants Ltd', 'longitude': '34.93809275', 'construction_year': '1999', 'installer': 'Roman', 'id': '69572', 'scheme_management': 'VWC', 'scheme_name': 'Roman', 'management': 'vwc', 'quantity_group': 'enough', 'source_class': 'groundwater', 'source': 'spring', 'subvillage': 'Mnyusi B', 'public_meeting': 'True', 'num_private': '0', 'latitude': '-9.85632177', 'waterpoint_type_group': 'communal standpipe', 'basin': 'Lake Nyasa', 'extraction_type_class': 'gravity', 'waterpoint_type': 'communal standpipe', 'wpt_name': 'none', 'management_group': 'user-group', 'lga': 'Ludewa', 'source_type': 'spring', 'district_code': '5', 'ward': 'Mundindi', 'payment': 'pay annually', 'population': '109', 'date_recorded': '2011-03-14', 'extraction_type_group': 'gravity', 'region': 'Iringa', 'amount_tsh': '6000.0', 'quality_group': 'good', 'payment_type': 'annually', 'water_quality': 'soft', 'permit': 'False

In [7]:
cleanData = quantData(rawData)
print cleanData[:1]

[{'extraction_type': 'gravity', 'gps_height': 1390, 'longitude': 34.93809275, 'construction_year': 1999.0, 'id': '69572', 'management': 'vwc', 'source_class': 'groundwater', 'source': 'spring', 'public_meeting': False, 'latitude': -9.85632177, 'basin': 'Lake Nyasa', 'extraction_type_class': 'gravity', 'waterpoint_type': 'communal standpipe', 'management_group': 'user-group', 'payment': 'pay annually', 'population': 109.0, 'date_recorded': 734119, 'extraction_type_group': 'gravity', 'region': 'Iringa', 'amount_tsh': 6000.0, 'quality_group': 'good', 'water_quality': 'soft', 'permit': False, 'quantity': 'enough'}]


In [10]:
vecData = vectorizeData(cleanData)
print vecData[:1]

[[  6.00000000e+03   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.99900000e+03   7.34119000e+05   1.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    0.00000000e+00   1.39000000e+03   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00  -9.85632177e+00
    3.49380928e+01   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   1.00000000e+00
    0.00000000e+00   0.00000000e+00   1.09000000e+02   0.00000000e+00
    1.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   1.00000000e+00   0.00000000e+00
    1.00000000e+00   1.00000000e+00   0.00000000e+00]]


In [14]:
vecData.shape

(5L, 51L)

In [1]:
def uniqueValues(data, field):
    #list of unique values for a particular field
    uniqueValues = []
    for row in data:
        if row[field] not in uniqueValues:
            uniqueValues.append(row[field])
    return uniqueValues

def uniquePairs(data, fld1, fld2):
    #list of unique pairs of fields - for finding redundant columns
    uniquePairs = []
    for row in data:
        if (row[fld1], row[fld2]) not in uniquePairs:
            uniquePairs.append((row[fld1], row[fld2]))
    return uniquePairs

In [5]:
data = getData()
print uniqueValues(data, 'water_quality')
print uniquePairs(data, 'water_quality', 'quality_group')

['soft', 'salty', 'milky', 'unknown', 'fluoride', 'coloured', 'salty abandoned', 'fluoride abandoned']
[('soft', 'good'), ('salty', 'salty'), ('milky', 'milky'), ('unknown', 'unknown'), ('fluoride', 'fluoride'), ('coloured', 'colored'), ('salty abandoned', 'salty'), ('fluoride abandoned', 'fluoride')]


In [6]:
print uniquePairs(data, 'source', 'source_type')

[('spring', 'spring'), ('rainwater harvesting', 'rainwater harvesting'), ('dam', 'dam'), ('machine dbh', 'borehole'), ('other', 'other'), ('shallow well', 'shallow well'), ('river', 'river/lake'), ('hand dtw', 'borehole'), ('lake', 'river/lake'), ('unknown', 'other')]


In [7]:
print uniquePairs(data, 'region', 'region_code')

[('Iringa', '11'), ('Mara', '20'), ('Manyara', '21'), ('Mtwara', '90'), ('Kagera', '18'), ('Tanga', '4'), ('Shinyanga', '17'), ('Tabora', '14'), ('Pwani', '60'), ('Ruvuma', '10'), ('Kilimanjaro', '3'), ('Rukwa', '15'), ('Mwanza', '19'), ('Kigoma', '16'), ('Lindi', '80'), ('Dodoma', '1'), ('Pwani', '6'), ('Arusha', '2'), ('Mbeya', '12'), ('Singida', '13'), ('Morogoro', '5'), ('Lindi', '18'), ('Dar es Salaam', '7'), ('Mtwara', '99'), ('Arusha', '24'), ('Mtwara', '9'), ('Mwanza', '17'), ('Lindi', '8'), ('Tanga', '5'), ('Shinyanga', '14'), ('Shinyanga', '11'), ('Pwani', '40')]


In [11]:
print uniquePairs(data, 'management', 'scheme_management')

[('vwc', 'VWC'), ('wug', 'Other'), ('other', ''), ('wug', ''), ('vwc', ''), ('private operator', 'Private operator'), ('wug', 'WUG'), ('water board', 'Water Board'), ('wua', 'WUA'), ('vwc', 'Water authority'), ('company', 'Company'), ('private operator', 'WUA'), ('other', 'Other'), ('water authority', 'Water authority'), ('private operator', ''), ('vwc', 'WUG'), ('private operator', 'Water authority'), ('parastatal', 'Parastatal'), ('vwc', 'Company'), ('wug', 'VWC'), ('water board', 'Water authority'), ('parastatal', 'VWC'), ('wug', 'WUA'), ('unknown', ''), ('private operator', 'VWC'), ('wua', 'VWC'), ('wug', 'Water Board'), ('vwc', 'Water Board'), ('other', 'VWC'), ('other', 'Water Board'), ('vwc', 'WUA'), ('other - school', 'VWC'), ('parastatal', 'Company'), ('private operator', 'Other'), ('vwc', 'Private operator'), ('wua', 'Water authority'), ('water board', 'VWC'), ('private operator', 'Company'), ('private operator', 'Parastatal'), ('vwc', 'Other'), ('vwc', 'Parastatal'), ('wua',

In [10]:
print uniqueValues(data, 'quantity')

['enough', 'insufficient', 'dry', 'seasonal', 'unknown']
